# Study – Glass – Stretcher: bissection

In [41]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

In [42]:
# For type annotations
import numpy as np

In [43]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [44]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

### Printing What the Model Predicted

_Aka_ printing the network output.

In [45]:
i = 0
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 0
1/1 [==============================] - 0s 35ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.56761974
Network output: (<class 'numpy.int64'>) 0


### Printing the Minimal Explanation

Minimal eplanations only indicates which inputs are relevant to get to a conclusion.

**Note:** The explanation happens _after_ the keras_model make its predictions.

In [46]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,-0.2792306037275416),
 docplex.mp.LinearConstraint[input2](x_1,EQ,-0.267930022273471),
 docplex.mp.LinearConstraint[input3](x_2,EQ,0.6074173914551261),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-0.8032068868488967),
 docplex.mp.LinearConstraint[input5](x_4,EQ,0.8170760979829164),
 docplex.mp.LinearConstraint[input6](x_5,EQ,0.0779031564307683),
 docplex.mp.LinearConstraint[input7](x_6,EQ,-0.4927117463954317),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

### Trying to Improve the Explanation

Given a minimal explanation, can we improve it?

Constraints of type $x = c$ are equivalent to $x \le c \land x \ge c$.

Therefore, we need to substitute each $x = c$ constraint by the $x \le c$ and $x \ge c$ constraints.

Then, we try stretching the interval by substituting $x \le c$ by $x \le c + \Delta x$ and see if our prediction changes. If the prediction stays the same, then we substitue and try stretching it again. If the prediction changes, then this new interval isn't valid and we don't substitute. We found the upper bound of the interval, i.e. $x \le c$.

Then we try to stretch the interval to fin the lower bound. Analogously, We try substituting $c \ge x$ by ????/


We will end up with a pair of constraints the looks like $c - k_l \cdot \Delta{x} \le x$ and $x \le c + k_u \cdot \Delta{x}$, i.e. this pair represents $c - k_l \cdot \Delta{x} \le x \le c + k_u \cdot \Delta{x}$.

### Setting Up

In [47]:
import docplex

In [48]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

#### Quick Sratch

In [49]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,-0.2792306037275416),
 docplex.mp.LinearConstraint[input2](x_1,EQ,-0.267930022273471),
 docplex.mp.LinearConstraint[input3](x_2,EQ,0.6074173914551261),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-0.8032068868488967),
 docplex.mp.LinearConstraint[input5](x_4,EQ,0.8170760979829164),
 docplex.mp.LinearConstraint[input6](x_5,EQ,0.0779031564307683),
 docplex.mp.LinearConstraint[input7](x_6,EQ,-0.4927117463954317),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [50]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [51]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,-0.2792306037275416),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-0.2792306037275416),
 docplex.mp.LinearConstraint[input LE](x_1,LE,-0.267930022273471),
 docplex.mp.LinearConstraint[input GE](x_1,GE,-0.267930022273471),
 docplex.mp.LinearConstraint[input LE](x_2,LE,0.6074173914551261),
 docplex.mp.LinearConstraint[input GE](x_2,GE,0.6074173914551261),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-0.8032068868488967),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-0.8032068868488967),
 docplex.mp.LinearConstraint[input LE](x_4,LE,0.8170760979829164),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.8170760979829164),
 docplex.mp.LinearConstraint[input LE](x_5,LE,0.0779031564307683),
 docplex.mp.LinearConstraint[input GE](x_5,GE,0.0779031564307683),
 docplex.mp.LinearConstraint[input LE](x_6,LE,-0.4927117463954317),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-0.4927117463954317),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292

In [52]:
def get_x_ge_value_constraints(linear_constraints):
	# x >= c
	# c <= x
	ge_constraints = []
	for xp in linear_constraints:
		if xp.sense == docplex.mp.constants.ComparisonType.GE:
			ge_constraints.append(xp)
	return ge_constraints

In [53]:
def get_x_le_value_constraints(linear_constraints):
	# x <= c
	le_constraints = []
	for xp in linear_constraints:
		if xp.sense == docplex.mp.constants.ComparisonType.LE:
			le_constraints.append(xp)
	return le_constraints

##### The Algorithm

In [54]:
def print_info(constraint):
	x = constraint.lhs
	print(f'{x.lb} <= {x} <= {x.ub}')

In [55]:
def print_c(constraint):
	x = constraint.lhs
	c = constraint.rhs.constant
	if constraint.sense == docplex.mp.constants.ComparisonType.GE:
		print(f'{constraint.name}: {c} <= {x}')
	# elif constraint.sense == docplex.mp.constants.ComparisonType.LE:
	# 	print(constraint)
	else:
		print(constraint)

In [56]:
def print_status(constraint, delta, epsilon):
	print_info(constraint)
	print_c(constraint)
	print(f'{delta} = delta <= epsilon = {epsilon}' if delta <= epsilon else f'{delta} = delta >= epsilon = {epsilon}')

In [57]:
def try_to_walk(docplex_model, constraint, step: float, precision: float) -> bool:
	"""
	Returns `True` if should wak again, and `False` otherwise
	
	Possibly modifies `constraint`, and as consequence `docplex_model`
	"""
	docplex_model.solve()
	has_solution = (docplex_model.solution == None)

	is_precise_enough = step <= precision/2
	if has_solution and is_precise_enough:
		return False

	if constraint.sense == docplex.mp.constants.ComparisonType.GE:
		# x >= c
		# c <= x
		if has_solution:
			# Walk left
			constraint.rhs.constant -= step
		else:
			# Walk right
			constraint.rhs.constant += step
	elif constraint.sense == docplex.mp.constants.ComparisonType.LE:
		# x <= c
		if has_solution:
			# Walk right
			constraint.rhs.constant += step
		else:
			# Walk left
			constraint.rhs.constant -= step
	else:
		raise NotImplementedError


	return True

In [58]:
def strecth(docplex_model, constraint, precision: float):
	"""
	Possibly mutates `constraint` and `docplex_model`
	"""
	x = constraint.lhs
	c = constraint.rhs.constant
	print(x)
	print(c)
	if constraint.sense == docplex.mp.constants.ComparisonType.GE:
		# x >= c
		# c <= x
		delta = c - x.lb
	elif constraint.sense == docplex.mp.constants.ComparisonType.LE:
		# x <= c
		delta = x.ub - c
	else:
		return

	assert delta >= 0, 'delta was negative'
	
	did_walk = True
	while did_walk:
		did_walk = try_to_walk(docplex_model, constraint, delta, precision)
		delta /= 2
		print(delta)

In [59]:
get_x_ge_value_constraints(linear_constraints)

[docplex.mp.LinearConstraint[input GE](x_0,GE,-0.2792306037275416),
 docplex.mp.LinearConstraint[input GE](x_1,GE,-0.267930022273471),
 docplex.mp.LinearConstraint[input GE](x_2,GE,0.6074173914551261),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-0.8032068868488967),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.8170760979829164),
 docplex.mp.LinearConstraint[input GE](x_5,GE,0.0779031564307683),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-0.4927117463954317),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [60]:
ct = get_x_ge_value_constraints(linear_constraints)[6]
strecth(testing_model, ct, 0.01)
ct

x_6
-0.4927117463954317
0.9897705450937662
0.4948852725468831
0.24744263627344154
0.12372131813672077
0.061860659068360385
0.030930329534180193
0.015465164767090096
0.007732582383545048
0.003866291191772524
0.001933145595886262


docplex.mp.LinearConstraint[input GE](x_6,GE,-4.444061344386951)

In [21]:
epsilon = 0.01

variables = testing_model.find_matching_vars('x')

print('Initial constraints:')
print(get_x_ge_value_constraints(linear_constraints))
print(get_x_le_value_constraints(linear_constraints))

for constraint in linear_constraints:
	strecth(testing_model, constraint, precision=epsilon)
	print('FINAL:', constraint)

print()
print('Final constraints:')
print(get_x_ge_value_constraints(linear_constraints))
print(get_x_le_value_constraints(linear_constraints))

Initial constraints:
[docplex.mp.LinearConstraint[input GE](x_0,GE,-0.2792306037275416), docplex.mp.LinearConstraint[input GE](x_1,GE,-0.267930022273471), docplex.mp.LinearConstraint[input GE](x_2,GE,0.6074173914551261), docplex.mp.LinearConstraint[input GE](x_3,GE,-3.8621150334423695), docplex.mp.LinearConstraint[input GE](x_4,GE,0.8170760979829164), docplex.mp.LinearConstraint[input GE](x_5,GE,0.0779031564307683), docplex.mp.LinearConstraint[input GE](x_6,GE,-0.4927117463954317), docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898), docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]
[docplex.mp.LinearConstraint[input LE](x_0,LE,-0.2792306037275416), docplex.mp.LinearConstraint[input LE](x_1,LE,-0.267930022273471), docplex.mp.LinearConstraint[input LE](x_2,LE,0.6074173914551261), docplex.mp.LinearConstraint[input LE](x_3,LE,-0.8032068868488967), docplex.mp.LinearConstraint[input LE](x_4,LE,0.8170760979829164), docplex.mp.LinearConstraint[input LE](x_5,LE,0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


**TODO:** Rewrite pair of expression of type $x \le c$ and $x \ge c$ to $x = c$:

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

Note that `x_6` is actually equal to `4.24127975754059`

In [ ]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [ ]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

### Pretty Printing the Explanation

In [ ]:
def get_variable_index(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [ ]:
def print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		variable = e.lhs
		index = get_variable_index(variable)
		feature_name = dataframe.columns[index]
		print(feature_name, e.sense.operator_symbol, e.rhs)

In [ ]:
print_explanation(improved_explanation)

## Comparing with Anchor

In [ ]:
from anchor import utils

### Loading the Dataset

In [ ]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	# categorical_features=None,
	# features_to_use=None,
	# feature_transformations=None,
	# discretize=False,
	# balance=False,
	# fill_na='-1',
	# filter_fn=None,
	skip_first=True
)

### Explainer

In [ ]:
from anchor import anchor_tabular

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names
)

In [ ]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [ ]:
for a in d.train:
	a == data[i]

In [ ]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

In [ ]:
exp.names()

In [ ]:
print_explanation(improved_explanation)

In [ ]:
exp.precision()

In [ ]:
exp.coverage()

In [ ]:
# explainer.explain_instance(
# 	data_row,
# 	classifier_fn,
# 	threshold=0.95,
# 	delta=0.1,
# 	tau=0.15,
# 	batch_size=100,
# 	max_anchor_size=None,
# 	desired_label=None,
# 	**kwargs)